In [3]:
import os
from PIL import Image, ImageFile
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np

ImageFile.LOAD_TRUNCATED_IMAGES = True  # 이미지 파일이 손상되었을 때 에러 발생 방지

def get_imglist(dir="./sample/img"):
    imglist = [os.path.join(dir, f).replace("\\", "/") for f in os.listdir(dir) if f.endswith('.png')]
    return imglist

class ImagePatchDataset(Dataset):
    def __init__(self, image, crop_size, transform=None):
        self.image = image
        self.crop_size = crop_size
        self.transform = transform
        self.image_width, self.image_height = image.size

        self.num_patches_x = (self.image_width + self.crop_size - 1) // self.crop_size
        self.num_patches_y = (self.image_height + self.crop_size - 1) // self.crop_size
        self.total_patches = self.num_patches_x * self.num_patches_y

    def __len__(self):
        return self.total_patches

    def __getitem__(self, idx):
        i = idx % self.num_patches_x
        j = idx // self.num_patches_x
        top_left_x = i * self.crop_size
        top_left_y = j * self.crop_size
        bottom_right_x = min(top_left_x + self.crop_size, self.image_width)
        bottom_right_y = min(top_left_y + self.crop_size, self.image_height)

        if bottom_right_x - top_left_x < self.crop_size:
            top_left_x = max(self.image_width - self.crop_size, 0)
            bottom_right_x = self.image_width
        if bottom_right_y - top_left_y < self.crop_size:
            top_left_y = max(self.image_height - self.crop_size, 0)
            bottom_right_y = self.image_height

        cropped_image = self.image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))

        resize = self.crop_size * 2
        cropped_image = cropped_image.resize((resize, resize), resample=Image.Resampling.LANCZOS)

        cropped_image_np = np.array(cropped_image)
        cropped_image_bgr = cropped_image_np[:, :, ::-1]
        cropped_image_pil = Image.fromarray(cropped_image_bgr, 'RGB')

        if self.transform:
            cropped_image = self.transform(cropped_image_pil)
        else:
            cropped_image = transforms.ToTensor()(cropped_image_pil)

        position = torch.tensor([top_left_x, top_left_y])

        return {'image': cropped_image, 'position': position}


/Users/seungwoo/anaconda3/envs/dust/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from torchvision import transforms
# import numpy as np
# import cv2
# import glob

# ImageFile.LOAD_TRUNCATED_IMAGES = True  # 이미지 파일이 손상되었을 때 에러 발생 방지
# # 이미지 목록을 가져오는 함수
# def get_imglist(dir="./sample/img"):
#     imglist = [os.path.join(dir, f).replace("\\", "/") for f in os.listdir(dir) if f.endswith('.png')]
#     return imglist

# class CroppedImageDataset(Dataset):
#     def __init__(self, image_list, crop_size):
#         self.image_list = image_list
#         self.crop_size = crop_size
#         self.transform = transforms.ToTensor()  # 이미지 -> 텐서 변환

#     def __len__(self):
#         return len(self.image_list)

#     def __getitem__(self, idx):
#         image_path = self.image_list[idx]
#         image_name = os.path.basename(image_path)

#         # 이미지 열기
#         image = Image.open(image_path).convert('RGB')
#         image_width, image_height = image.size
#         # # 이미지를 BGR로 변환
#         image_np = np.array(image)  # PIL 이미지 -> NumPy 배열
#         image_bgr = image_np[:, :, ::-1]  # RGB -> BGR로 색상 채널 순서 변경
#         image = Image.fromarray(image_bgr, 'RGB')  # NumPy 배열 -> PIL 이미지
#         # 전체 크롭 이미지 개수 계산
#         num_crops_x = (image_width + self.crop_size - 1) // self.crop_size
#         num_crops_y = (image_height + self.crop_size - 1) // self.crop_size
#         total_crops = num_crops_x * num_crops_y

#         # 크롭할 영역의 좌상단 좌표를 슬라이딩 윈도우 방식으로 구함
#         cropped_images = []
#         positions = []
#         last_cropped_image_info = None  # 마지막 크롭된 이미지 정보 저장

#         for top_left_x in range(0, image_width, self.crop_size):
#             for top_left_y in range(0, image_height, self.crop_size):
#                 # 마지막 부분에서 경계 넘지 않도록 마지막 부분을 맞춤
#                 bottom_right_x = min(top_left_x + self.crop_size, image_width)
#                 bottom_right_y = min(top_left_y + self.crop_size, image_height)

#                 # 이미지 경계 부분에 대해 크롭 영역을 이동시킴
#                 if bottom_right_x - top_left_x < self.crop_size:
#                     top_left_x = image_width - self.crop_size
#                     bottom_right_x = image_width

#                 if bottom_right_y - top_left_y < self.crop_size:
#                     top_left_y = image_height - self.crop_size
#                     bottom_right_y = image_height

#                 # 크롭한 이미지 자르기
#                 cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
#                 ############################################################
#                 # 크롭한 이미지를 resize
#                 resize = self.crop_size * 2
#                 cropped_image = cropped_image.resize((resize, resize), resample=Image.Resampling.LANCZOS)
#                 ############################################################
#                 # 크롭한 이미지를 텐서로 변환
#                 cropped_image_tensor = self.transform(cropped_image)

#                 # 크롭한 이미지와 좌상단 좌표 저장
#                 cropped_images.append(cropped_image_tensor)
#                 positions.append(torch.tensor([top_left_x, top_left_y]))

#                 # 마지막 크롭된 이미지의 정보 저장 (좌표와 실제 크기)
#                 last_cropped_image_info = {
#                     'image_tensor': cropped_image_tensor,
#                     'top_left': (top_left_x, top_left_y),
#                     'bottom_right': (bottom_right_x, bottom_right_y),
#                     'size': (bottom_right_x - top_left_x, bottom_right_y - top_left_y)  # 실제 크기 저장
#                 }

#         # 이미지 이름, 크롭한 이미지 텐서 목록, 각 이미지의 좌상단 좌표 및 크롭 개수 반환
#         return {
#             'image_name': image_name,
#             'images': cropped_images,  # 잘라낸 이미지 텐서 리스트
#             'top_left_positions': positions,  # 각 이미지의 좌상단 좌표 리스트
#             'total_crops': total_crops,  # 총 크롭 이미지 개수
#             'last_cropped_image_info': last_cropped_image_info  # 마지막 크롭 이미지 정보
#         }
# # 배치 데이터를 처리하는 collate_fn 정의
# def collate_fn(batch, batch_size):
#     all_image_names = []
#     all_images = []
#     all_top_left_positions = []
#     total_crops = 0  # 전체 크롭 이미지 개수를 추적
#     last_cropped_images_info = []  # 마지막 크롭 이미지 정보 추적

#     for item in batch:
#         image_names = [item['image_name']] * len(item['images'])  # 각 이미지에 같은 이름을 붙임
#         all_image_names.extend(image_names)
#         all_images.extend(item['images'])  # 이미지를 리스트에 추가
#         all_top_left_positions.extend(item['top_left_positions'])  # 좌상단 좌표 추가
#         total_crops += item['total_crops']  # 총 크롭 개수 계산
#         last_cropped_images_info.append(item['last_cropped_image_info'])  # 마지막 크롭 정보 추가

#     # 전체 이미지 목록을 batch_size 크기씩 나눠서 반환
#     batch_start = 0
#     while batch_start < len(all_images):
#         images_batch = torch.stack(all_images[batch_start:batch_start + batch_size])  # batch_size만큼 이미지 묶기
#         positions_batch = torch.stack(all_top_left_positions[batch_start:batch_start + batch_size])  # batch_size만큼 좌표 묶기
#         names_batch = all_image_names[batch_start:batch_start + batch_size]  # batch_size만큼 이미지 이름 묶기
        
#         batch_start += batch_size
        
#         yield {
#             'image_names': names_batch,  # 이미지 이름 리스트
#             'images': images_batch,  # [batch_size, 3, crop_size, crop_size]
#             'top_left_positions': positions_batch,  # [batch_size, 2]
#             'total_crops': total_crops,  # 전체 크롭 이미지 개수
#             'last_cropped_images_info': last_cropped_images_info  # 마지막 크롭 이미지 정보
#         }


In [ ]:
from ultralytics import YOLO
import torch
# 사용 예시
#directory_path = '/workspace/dataset/'
directory_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
])

crop_size = 224
batch_size = 2
img_list = get_imglist(directory_path)

# color_stats_file = './train_color_stats.npz'

####################
model_name = "./yolo11s-obb.pt"
####################
# 모델 정의 
model = YOLO(model_name)  # YOLO OBB 모델 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "mps")  # GPU 또는 MPS 사용

# 모델은 이미 내부적으로 GPU/MPS를 사용하므로 입력 이미지를 device로 보냄
# dataset = CroppedImageDataset(img_list, crop_size)
# dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=lambda x: collate_fn(x, batch_size))



for image_path in img_list:
    image_name = os.path.basename(image_path)
    print(f"Processing image: {image_name}")

    image = Image.open(image_path).convert('RGB')

    patch_dataset = ImagePatchDataset(image, crop_size, transform=transform)
    patch_loader = DataLoader(patch_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

    for batch in patch_loader:
        images = batch['image']
        positions = batch['position']

    del image

Processing image: task_smaple.png


/Users/seungwoo/anaconda3/envs/dust/lib/python3.12/site-packages/PIL/Image.py:3368: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/seungwoo/anaconda3/envs/dust/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/seungwoo/anaconda3/envs/dust/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'ImagePatchDataset' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


In [ ]:


# YOLO 모델을 사용한 예측 함수
def run_yolo_on_images(dataloader, model, device):
    results = []  # 예측 결과를 저장할 리스트
    
    for batch in dataloader:
        for sub_batch in batch:  # 각 sub_batch에 대해 처리
            images = sub_batch['images'].to(device)  # 이미지를 device로 전송
            top_left_positions = sub_batch['top_left_positions'].to(device)  # 좌상단 좌표도 device로 전송
            
            # YOLO 모델 예측 수행
            preds = model.predict(images,conf=0.1,save=True)  # Ultralytics YOLO 모델의 predict 함수 사용

            obb = [ pred.obb.xywhr for pred in preds]
            conf = [ pred.obb.conf for pred in preds]
            results.append({
                'image_names': sub_batch['image_names'],
                'obb':obb,
                'conf':conf,
                'top_left_positions': top_left_positions
                
            })
            print(f"Processed {len(sub_batch['image_names'])} images with predictions.")
    
    return results

# 모델을 사용한 예측 수행
predictions = []
predictions += run_yolo_on_images(dataloader, model, device)
print("예측 완료")
print(len(predictions))

In [6]:
import csv
import torch
from torchvision.ops import nms
import math
import datetime
from ultralytics.utils.ops import nms_rotated

# END: Add timestamp to the CSV filename
# 파일 저장할 CSV 경로
csv_file = "./submission.csv"

# 데이터 샘플 (image_name, cx, cy, width, height, angle 등)
data = []


# NMS 임계값 (IoU 임계값)
nms_threshold = 0.7

# 'predictions' 리스트에 있는 각 배치에서 데이터를 추출
for i in range(len(predictions)):  # predictions 리스트에서 하나씩 꺼냄
    image_name = predictions[i]['image_names']  # 각 배치의 이미지 이름 리스트
    obb_list = predictions[i]['obb']  # 각 배치의 obb 리스트
    top_left_pos_list = predictions[i]['top_left_positions']  # 각 배치의 top_left_positions 리스트
    conf_list = predictions[i]['conf']  # 각 배치의 conf 리스트
    # 각 배치에서 이미지별로 순회
    for j in range(len(obb_list)):
        obb_tensor = obb_list[j]
        top_left_pos = top_left_pos_list[j]
        conf = conf_list[j]
        
        # obb_tensor가 비어있지 않은 경우에만 처리
        if len(obb_tensor) > 0:
            # NMS 처리를 위한 준비
            boxes = []
            scores = [conf[k].item() for k in range(len(conf))]
            
            for k in range(len(obb_tensor)):
                cx = obb_tensor[k][0].item()/2 + top_left_pos[0].item()
                cy = obb_tensor[k][1].item()/2 + top_left_pos[1].item()
                width = obb_tensor[k][2].item()/2
                height = obb_tensor[k][3].item()/2
                angle = obb_tensor[k][4].item()

                # 사각형 좌표로 변환 (cx, cy, width, height -> x1, y1, x2, y2)
                x1 = cx - width / 2
                y1 = cy - height / 2
                x2 = cx + width / 2
                y2 = cy + height / 2

                # 박스와 점수 추가
                boxes.append([x1, y1, x2, y2])
                

                # NMS 수행
                boxes_tensor = torch.tensor(boxes, dtype=torch.float32)
                scores_tensor = torch.tensor(scores, dtype=torch.float32)
                nms_indices = nms(boxes_tensor, scores_tensor, nms_threshold)
                print("nms indices 완료.")
                print(len(nms_indices))
                
                # NMS 후 남은 객체들에 대해 각도 변환 및 데이터 추가
            for idx in nms_indices:
                cx = obb_tensor[idx][0].item()/2 + top_left_pos[0].item()
                cy = obb_tensor[idx][1].item()/2 + top_left_pos[1].item()
                width = obb_tensor[idx][2].item()/2
                height = obb_tensor[idx][3].item()/2
                angle = obb_tensor[idx][4].item()
                
                # 라디안을 도 단위로 변환
                angle_deg = math.degrees(angle)
                
                # 각도를 0~360도 범위로 변환
                if angle_deg < 0:
                    angle_deg += 360

                # 데이터 추가
                data.append([image_name[j], cx, cy, width, height, angle_deg])
# CSV 파일로 저장
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # CSV의 헤더 작성
    writer.writerow(['image_name', 'cx', 'cy', 'width', 'height', 'angle'])
    
    # 각 행을 작성
    writer.writerows(data)

print(f"CSV 파일 '{csv_file}'이(가) 성공적으로 생성되었습니다.")


CSV 파일 './submission.csv'이(가) 성공적으로 생성되었습니다.
